Andri Heru Saputra (18917206)
<br/>
Yuan Sa'adati (18917226)
<br />
Tugas 1 : Data Sequence dengan Metode LSTM

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re


Using TensorFlow backend.


Data = https://www.kaggle.com/crowdflower/twitter-airline-sentiment/home#Tweets.csv
<br/>
tentang sentimen analisis terhadap maskapai US di twitter

In [0]:
data = pd.read_csv('/content/Tweets.csv')
# Keeping only the neccessary columns
data = data[['text', 'airline_sentiment']]

Selanjutnya, kami mengenyampingkan sentimen 'Netral' karena tujuan kami hanya membedakan tweet positif dan negatif. Setelah itu, kami memfilter tweet sehingga hanya teks dan kata-kata yang valid yang tersisa. Kemudian, kami menentukan jumlah fitur maks adalah 2000 dan menggunakan Tokenizer untuk membuat vektor dan mengubah teks menjadi Sequence sehingga akan mudah diinputkan.

In [3]:
data = data[data.airline_sentiment != "neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['airline_sentiment'] == 'positive'].size)
print(data[ data['airline_sentiment'] == 'negative'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

4726
18356


Selanjutnya, kami menganalisa dengan LSTM. Perhatikan bahwa variabel embed_dim, lstm_out, batch_size, droupout_x adalah hiperparameter. Kami menggunakan softmax sebagai fungsi aktivasi. Alasannya adalah bahwa Jaringan kami menggunakan category_crossentropy, dan softmax adalah metode aktivasi yang tepat untuk itu.

In [4]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 32, 128)           256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 32, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
Y = pd.get_dummies(data['airline_sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(7732, 32) (7732, 2)
(3809, 32) (3809, 2)


dengan epoch = 7 dan batch_size = 32 maka didapatkan nilai akurasi terbaiknya = 0.9737 dengan nilai loss = 0.0684

In [7]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/7





 - 22s - loss: 0.3310 - acc: 0.8639
Epoch 2/7
 - 21s - loss: 0.1888 - acc: 0.9283
Epoch 3/7
 - 21s - loss: 0.1499 - acc: 0.9435
Epoch 4/7
 - 21s - loss: 0.1224 - acc: 0.9538
Epoch 5/7
 - 21s - loss: 0.1000 - acc: 0.9639
Epoch 6/7
 - 21s - loss: 0.0838 - acc: 0.9693
Epoch 7/7
 - 21s - loss: 0.0684 - acc: 0.9737


In [8]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.34
acc: 0.91
